In [1]:
import pandas as pd
import numpy as np

In [2]:
# %load get_data.py
def get_data() :
    
    import pandas as pd
    
    csv_beer = pd.read_csv("/home/grimoire/Projects/BeerRatings/rating_update.csv")
    beer_ratings = pd.DataFrame(csv_beer)
    
    return beer_ratings


In [3]:
ratings = get_data()

In [4]:
test = ratings.copy()

In [5]:
test.keys()

Index(['brewery_id', 'brewery_name', 'review_time', 'review_overall',
       'review_aroma', 'review_appearance', 'review_profilename', 'beer_style',
       'review_palate', 'review_taste', 'beer_name', 'beer_abv',
       'beer_beerid'],
      dtype='object')

In [6]:
test = test.drop(['brewery_id', 'brewery_name', 'review_time', 'review_profilename', 
                  'beer_style', 'beer_abv','beer_beerid'], axis=1)

In [7]:
total_sums = test.review_overall.values + test.review_aroma.values + \
             test.review_appearance.values + test.review_palate + test.review_taste

In [8]:
% timeit test['combined'] = total_sums/5

3.49 ms ± 47 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
beer_grouping = test.groupby(['beer_name']).mean()

In [11]:
beer_grouping = beer_grouping.merge(test.groupby('beer_name').count()
                                    .drop(['review_overall',
                                           'review_aroma', 
                                           'review_appearance', 
                                           'review_palate',
                                           'review_taste'], axis=1)
                                    .rename(columns = {'combined':'count'}), left_index=True, right_index=True)

In [17]:
beer_grouping.iloc[100,:]

review_overall       4.25
review_aroma         4.25
review_appearance    4.00
review_palate        4.00
review_taste         4.50
combined             4.20
count                2.00
Name: 't Goeye Goet Dubbel, dtype: float64

In [26]:
C = beer_grouping['review_overall'].mean()
m = 3
#q_ratings = beer_grouping.copy().loc[beer_grouping['count'] >= m]

In [27]:
def weighted_rating(df, m=m, C=C):
    
    #########################################
    # Calculates and returns a weighted rating for specific feature
    # m is minimum votes/ratings required to be listed
    # C is the mean rating/vote across the whole dataframe
    # R is average rating/votes of feature
    # v is number of ratings/votes of feature
    #########################################
    
    v = df['count']
    R = df['review_overall']
    
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

In [28]:
q_ratings['weighted_overall'] = q_ratings.apply(weighted_rating, axis=1)

In [31]:
q_ratings = q_ratings.sort_values('weighted_combined', ascending=False)

In [32]:
q_ratings

,review_overall,review_aroma,review_appearance,review_palate,review_taste,combined,count,weighted_overall,weighted_combined
beer_name,,,,,,,,,
Rare D.O.S.,4.848485,4.757576,4.469697,4.803030,4.848485,4.745455,33,4.748997,4.651131
M Belgian-Style Barleywine,4.750000,4.785714,4.482143,4.803571,4.857143,4.735714,28,4.643997,4.627120
Armand'4 Oude Geuze Lente (Spring),4.730769,4.715385,4.523077,4.669231,4.730769,4.673846,65,4.683293,4.627069
Pliny The Younger,4.600000,4.723770,4.482787,4.612295,4.724590,4.628689,610,4.595373,4.623721
Trappist Westvleteren 12,4.617925,4.583333,4.454009,4.633255,4.718553,4.601415,1272,4.615658,4.599091
Vanilla Bean Aged Dark Lord,4.476974,4.717105,4.450658,4.674342,4.710526,4.605921,152,4.461057,4.586714
Founders CBS Imperial Stout,4.591052,4.558085,4.457614,4.579278,4.697017,4.576609,637,4.586662,4.572095
Cantillon Blåbær Lambik,4.628205,4.528846,4.644231,4.493590,4.628205,4.584615,156,4.609836,4.566294
Dirty Horse,4.820513,4.615385,4.423077,4.576923,4.743590,4.635897,39,4.737236,4.562874


In [33]:
profile_ratings = pd.DataFrame(
                    pd.read_csv('/home/grimoire/Projects/BeerRatings/comparison_profile.csv'))

In [35]:
profile_ratings.total_profile_reviews.mean()

47.511486506724175